In [1]:
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
import joblib  


In [2]:
# MNIST Datensatz aus keras laden
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Das Shape des Datensatzes ausgeben
print("Ursprungs Shape des Trainingsdatensatzes:", x_train.shape)
print("Ursprungs Shape des Testdatensatzes:", x_test.shape)

Ursprungs Shape des Trainingsdatensatzes: (60000, 28, 28)
Ursprungs Shape des Testdatensatzes: (10000, 28, 28)


In [4]:
# Den Datensatz für ein SVM vorbereiten

# Flatten der Bildmatrizen für die SVM (von 28x28 zu 784)
x_train_svm = x_train.reshape((x_train.shape[0], -1))
x_test_svm = x_test.reshape((x_test.shape[0], -1))
# Standardisierung der Feature-Werte
scaler = StandardScaler()
x_train_svm = scaler.fit_transform(x_train_svm)
x_test_svm = scaler.transform(x_test_svm)

In [5]:
# Das Shape des Datensatzes ausgeben
print("SVM Shape des Trainingsdatensatzes:", x_train_svm.shape)
print("SVM Shape des Testdatensatzes:", x_test_svm.shape)

SVM Shape des Trainingsdatensatzes: (60000, 784)
SVM Shape des Testdatensatzes: (10000, 784)


In [6]:
# Parameteroptimierung mit RandomizedSearchCV

# Parameterverteilungen statt eines Gitters
param_distributions = {
    'C': [0.1, 1],
    'gamma': [0.1, 0.01]
    #'kernel': ['rbf']
}

# RandomizedSearchCV initialisieren
random_search = RandomizedSearchCV(svm.SVC(), param_distributions, n_iter=10, verbose=3, cv=3, random_state=42, n_jobs=-1)

# Auf den Trainingsdaten fitten
random_search.fit(x_train_svm, y_train)

# Beste Parameter-Kombination
print("Beste Parameter-Kombination:", random_search.best_params_)


c:\Users\micha\CODE\ml_algorithms\venv_ml\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 4 candidates, totalling 12 fits
Beste Parameter-Kombination: {'gamma': 0.01, 'C': 1}


In [10]:
import pandas as pd

# Save cv_results_df to a CSV file
cv_results_df.to_csv('svm_v1_opt_randcv_results.csv', index=False)
cv_results_df = pd.DataFrame(random_search.cv_results_)
print(cv_results_df)

   mean_fit_time  std_fit_time  mean_score_time  std_score_time param_gamma  \
0   13437.239037     11.404894      3674.571422        5.087596         0.1   
1    8724.611095     27.272017      3309.736212        4.505955        0.01   
2   14876.417873   3446.168224      2163.008726      900.058493         0.1   
3    5259.437868    996.988902      1581.440059      525.641913        0.01   

  param_C                     params  split0_test_score  split1_test_score  \
0     0.1   {'gamma': 0.1, 'C': 0.1}            0.11235            0.11235   
1     0.1  {'gamma': 0.01, 'C': 0.1}            0.60450            0.60685   
2       1     {'gamma': 0.1, 'C': 1}            0.17660            0.18110   
3       1    {'gamma': 0.01, 'C': 1}            0.82680            0.83315   

   split2_test_score  mean_test_score  std_test_score  rank_test_score  
0            0.11240         0.112367        0.000024                4  
1            0.61235         0.607900        0.003290              

In [18]:
# Das SVM Modell definieren
svc = svm.SVC(gamma=0.01, class_weight='balanced', C=1, kernel='rbf')

In [19]:
# Das Modell trainieren

svc.fit(x_train_svm, y_train)

In [13]:
# 7. Das SVM Modell auf den Testdaten evaluieren

result = svc.predict(x_test_svm)

print(classification_report(y_test, result))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       980
           1       1.00      0.72      0.84      1135
           2       1.00      0.00      0.00      1032
           3       0.11      1.00      0.20      1010
           4       1.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       1.00      0.00      0.01       958
           7       1.00      0.02      0.05      1028
           8       0.00      0.00      0.00       974
           9       1.00      0.01      0.02      1009

    accuracy                           0.19     10000
   macro avg       0.61      0.18      0.11     10000
weighted avg       0.63      0.19      0.12     10000



c:\Users\micha\CODE\ml_algorithms\venv_ml\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\micha\CODE\ml_algorithms\venv_ml\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\micha\CODE\ml_algorithms\venv_ml\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [ ]:
# Auswertung

cm = confusion_matrix(y_test, result)

plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')
plt.show()

In [ ]:
# Plotting some true predictions

ig, axes = plt.subplots(3, 3, figsize=(4, 4))
fig.subplots_adjust(hspace=0.8, wspace=0.5)  # Adjust the spacing between images
for i, ax in enumerate(axes.flat):
    ax.imshow(x_test[i], cmap='gray')
    ax.set_title(f'Truth: {y_test[i]}\nPredicted: {result[i]}')
    ax.axis('off')
plt.show()

In [ ]:
# Plotting some false predictions

count = 0
mismatched_entries = []
for i in range(len(result)):
    if result[i] != y_test[i]:
        mismatched_entries.append((result[i], y_test[i], x_test[i]))
        count += 1
    if count == 9:
        break

fig, axes = plt.subplots(3, 3, figsize=(4, 4))
fig.subplots_adjust(hspace=0.8, wspace=0.5)  # Adjust the spacing between images
for i, ax in enumerate(axes.flat):
    ax.imshow(mismatched_entries[i][2], cmap='gray')
    ax.set_title(f'Truth: {mismatched_entries[i][1]}\nPredicted: {mismatched_entries[i][0]}')
    ax.axis('off')
plt.show()

In [ ]:
# Speichern des Modells

joblib.dump(svc, 'models/svm_v1_model.pkl')